In [1]:
%load_ext autoreload
%autoreload 2

In [93]:
import torch
from tqdm import tqdm_notebook as tqdm
test_dir = '../../datasets/xview/test'

In [115]:
from datetime import datetime
import os
submission_dir = f"../submissions/{str(datetime.now()).replace(' ', '_')}"
os.mkdir(submission_dir)

In [163]:
import yaml
class Config:
    def __init__(self, file_path, file_type='json'):
        with open(file_path) as f:
            if file_path.endswith('.json'):
                _conf = json.load(f)
            elif file_path.endswith('.yaml'):
                _conf = yaml.load(f)
        for k,v in _conf.items():
            setattr(self, k, v)

In [27]:
import json
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn
from xv.nn.layers import FrozenBatchNorm2d

def load_segmentation_model(conf_file, state_file):
    conf = Config(conf_file)

    segmentation_types = {
        'PSPNet': smp.PSPNet,
        'FPN': smp.FPN,
        'Linknet': smp.Linknet,
        'Unet': smp.Unet
    }

    model_classes = conf.nclasses

    model = segmentation_types[conf.segmentation_arch](
        conf.encoder,
        classes=model_classes,
        activation='sigmoid',
        attention_type=conf.attention
    )


    if conf.freeze_encoder_norm:
        model.encoder = FrozenBatchNorm2d.convert_frozen_batchnorm(model.encoder)

    if conf.freeze_decoder_norm:
        model.decoder = FrozenBatchNorm2d.convert_frozen_batchnorm(model.decoder)

    preprocess_fn = get_preprocessing_fn(conf.encoder)

    state_dict = torch.load(state_file)

    model.load_state_dict(state_dict)
    return model, preprocess_fn

In [29]:
conf_file = "../weights/unet-se-1024.json"
state_file = "../weights/unet-se-1024.pth"

model, preprocess_fn = load_segmentation_model(conf_file, state_file)
model = model.eval().cuda()

In [16]:
import ttach as tta

model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')

In [35]:
from glob import glob
pre_files = glob(f"{test_dir}/images/*pre*")

In [75]:
from PIL import Image
import numpy as np
import torch

def load_img(img_path, preprocess_fn):
    image = np.array(Image.open(img_path))
    image = preprocess_fn(image)
    image = image.transpose(2,0,1)
    image = image.astype(np.float32)
    return torch.Tensor(image[None])

In [122]:
with torch.no_grad():
    for f in tqdm(pre_files):
        i = load_img(f, preprocess_fn)
        out = model(i.cuda())[0][0] > 0
        out = out.cpu().numpy()
        fid = f.split('_')[-1].replace(".png", "")
        Image.fromarray(out).save(f"{submission_dir}/test_localization_{fid}_prediction.png")

In [ ]:
conf_file = "../weights/unet-se-1024.json"
state_file = "../weights/unet-se-1024.pth"

In [134]:
from xv.nn.nets import BoxClassifier

In [ ]:
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
backbone = resnet_fpn_backbone(conf.backbone, True)

if conf.pretrain_weights:
    state_dict = torch.load(conf.pretrain_weights)
    model.load_state_dict(state_dict)

In [168]:
dmg_conf_file = "../weights/damage_od_init.yaml"
dmg_state_file = "../weights/damage_od_init.pth"

In [178]:
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

def load_damage_model(conf_file, state_file):
    conf = Config(conf_file)
    backbone = resnet_fpn_backbone(conf.backbone, True)
    model = BoxClassifier(backbone, conf.nclasses)
    state_dict = torch.load(state_file)
    model.load_state_dict(state_dict)
    model = model.eval().cuda()
    return model

def load_dmg_img(img_path, image_mean = (0.485, 0.456, 0.406), image_std = (0.229, 0.224, 0.225)):
    image = np.array(Image.open(img_path))
    image = image.astype(np.float32)
    image /= 255.
    image = (image-image_mean)/image_std
    image = image.transpose(2,0,1)
    return torch.Tensor(image[None])

In [179]:
post_files = glob(f"{test_dir}/images/*post*")

In [297]:
for f in tqdm(post_files):
    fid = f.split('_')[-1].replace(".png", "")
    mask_file = f"{submission_dir}/test_localization_{fid}_prediction.png"
    msk = np.array(Image.open(mask_file))
    polys = Polygons.from_mask(msk)
    
    polypoints = polys.points
    if polypoints:
        with torch.no_grad():
            boxes = torch.Tensor([[min(p[:,0]), min(p[:,1]), max(p[:,0]), max(p[:,1])] for p in polypoints])
            inp = load_dmg_img(f)
            out = model(inp.cuda(), [boxes.cuda()])
            classes = (out.argmax(1) + 1).cpu().numpy()
    else:
        classes = []

    msk_dmg = np.zeros((1024, 1024), dtype=np.uint8)
    
    for poly, cls in zip(polypoints, classes):
        cv2.fillPoly(msk_dmg, [poly], int(cls))

    Image.fromarray(msk_dmg).save(f"{submission_dir}/test_damage_{fid}_prediction.png")

In [299]:
!ls {submission_dir} | wc -l

1866


In [314]:
!tar -czvf {submission_dir}.tar.gz {submission_dir}

tar: Removing leading `../' from member names
../submissions/2019-11-10_00:56:16.401490/
../submissions/2019-11-10_00:56:16.401490/test_damage_00758_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00842_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00609_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00073_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00178_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00371_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00312_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00496_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00576_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00458_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00276_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00

../submissions/2019-11-10_00:56:16.401490/test_damage_00727_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00592_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00465_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00846_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00738_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00025_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00623_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00164_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00357_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00756_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00521_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00644_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_0070

../submissions/2019-11-10_00:56:16.401490/test_localization_00448_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00621_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00673_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00021_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00901_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00415_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00222_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00304_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00910_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00886_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00143_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00649_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00494_pred

../submissions/2019-11-10_00:56:16.401490/test_localization_00375_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00646_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00509_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00347_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00033_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00458_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00902_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00628_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00883_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00367_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00280_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00123_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_0026

../submissions/2019-11-10_00:56:16.401490/test_localization_00264_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00744_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00536_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00423_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00507_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00714_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00476_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00190_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00476_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00110_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00307_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00055_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_0026

../submissions/2019-11-10_00:56:16.401490/test_damage_00926_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00917_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00348_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00549_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00512_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00767_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00569_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00722_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00822_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00370_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00641_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00068_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00267_pred

../submissions/2019-11-10_00:56:16.401490/test_damage_00209_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00188_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00243_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00819_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00063_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00867_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00246_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00512_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00556_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00576_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00765_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00177_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localizatio

../submissions/2019-11-10_00:56:16.401490/test_localization_00790_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00584_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00599_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00450_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00926_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00535_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00110_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00685_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00052_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00833_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00917_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00715_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localizatio

../submissions/2019-11-10_00:56:16.401490/test_damage_00691_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00005_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00239_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00372_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00321_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00259_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00031_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00046_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00309_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00053_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00090_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00862_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_local

../submissions/2019-11-10_00:56:16.401490/test_damage_00369_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00127_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00745_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00892_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00437_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00869_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00481_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00104_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00388_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00699_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_localization_00235_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00217_prediction.png
../submissions/2019-11-10_00:56:16.401490/test_damage_00376_pred

In [313]:
!zip -r {submission_dir}.zip {submission_dir}

updating: ../submissions/2019-11-10_00:56:16.401490/ (stored 0%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00758_prediction.png (deflated 4%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00842_prediction.png (deflated 3%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00609_prediction.png (deflated 64%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00073_prediction.png (deflated 3%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00178_prediction.png (deflated 19%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00371_prediction.png (deflated 8%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00312_prediction.png (deflated 18%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00496_prediction.png (deflated 1%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00576_prediction.png (deflated 47%)
  adding: 

  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00152_prediction.png (deflated 6%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00573_prediction.png (deflated 26%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00062_prediction.png (deflated 3%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00408_prediction.png (deflated 2%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00884_prediction.png (deflated 4%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00164_prediction.png (deflated 92%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00514_prediction.png (deflated 52%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00766_prediction.png (deflated 86%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00883_prediction.png (deflated 85%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00423_pred

  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00047_prediction.png (deflated 1%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00619_prediction.png (deflated 23%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00407_prediction.png (deflated 32%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00774_prediction.png (deflated 92%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00731_prediction.png (deflated 28%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00267_prediction.png (deflated 62%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00558_prediction.png (deflated 81%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00608_prediction.png (deflated 10%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00275_prediction.png (deflated 17%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00642_predict

  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00744_prediction.png (deflated 1%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00730_prediction.png (deflated 5%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00355_prediction.png (deflated 24%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00639_prediction.png (deflated 7%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00873_prediction.png (deflated 4%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00375_prediction.png (deflated 7%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00646_prediction.png (deflated 5%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00509_prediction.png (deflated 35%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00347_prediction.png (deflated 32%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00033_prediction.png (de

  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00682_prediction.png (deflated 70%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00067_prediction.png (deflated 7%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00838_prediction.png (deflated 84%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00211_prediction.png (deflated 86%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00227_prediction.png (deflated 4%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00900_prediction.png (deflated 35%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00605_prediction.png (deflated 28%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00854_prediction.png (deflated 16%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00882_prediction.png (deflated 30%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00

  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00735_prediction.png (deflated 22%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00026_prediction.png (deflated 83%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00751_prediction.png (deflated 8%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00064_prediction.png (deflated 92%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00190_prediction.png (deflated 56%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00419_prediction.png (deflated 92%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00790_prediction.png (deflated 38%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00825_prediction.png (deflated 2%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00003_prediction.png (deflated 81%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00897_prediction.png

  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00067_prediction.png (deflated 6%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00609_prediction.png (deflated 64%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00834_prediction.png (deflated 2%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00736_prediction.png (deflated 2%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00248_prediction.png (deflated 2%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00607_prediction.png (deflated 79%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00875_prediction.png (deflated 80%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00260_prediction.png (deflated 23%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00373_prediction.png (deflated 40%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00055_pred

  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00007_prediction.png (deflated 92%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00617_prediction.png (deflated 7%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00039_prediction.png (deflated 4%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00660_prediction.png (deflated 92%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00557_prediction.png (deflated 0%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00149_prediction.png (deflated 1%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00205_prediction.png (deflated 78%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00779_prediction.png (deflated 2%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00071_prediction.png (deflated 17%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00450_predi

  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00262_prediction.png (deflated 11%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00688_prediction.png (deflated 86%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00072_prediction.png (deflated 16%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00300_prediction.png (deflated 25%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00204_prediction.png (deflated 3%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00707_prediction.png (deflated 46%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00099_prediction.png (deflated 87%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00102_prediction.png (deflated 2%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_damage_00442_prediction.png (deflated 84%)
  adding: ../submissions/2019-11-10_00:56:16.401490/test_localization_00

In [315]:
!cd {submission_dir}

In [316]:
submission_dir

'../submissions/2019-11-10_00:56:16.401490'

In [317]:
!tar -czvf "2019-11-10_00:56:16.401490.tar.gz" 2019-11-10_00:56:16.401490

tar: 2019-11-10_00\:56\:16.401490: Cannot stat: No such file or directory
tar (child): Cannot connect to 2019-11-10_00: resolve failed
tar: Child returned status 128
tar: Error is not recoverable: exiting now


In [318]:
!ls

config	inference.ipynb  od-damage.ipynb  segment_building.ipynb  wandb
